In [13]:
import copy
import time
setCulori = set()
posibileCombinatii = []
combinatiiSiRezultat = []
stare_fin = []
stare_init = None
citesteDinFisierul = ""
# g = open("output.txt", "w")
# g = open("outputScurt.txt", "w")
# g = open("outputFaraSolutii.txt", "w")
# g = open("outputLaFel.txt", "w")
g = open("output2.txt", "w")

In [14]:
def combina_culori(culoare1, culoare2):
        ok = 1
        culoare_rezultata = ""
        for i in range(len(combinatiiSiRezultat)):
            if culoare1 == combinatiiSiRezultat[i][0] and culoare2 == combinatiiSiRezultat[i][1] or culoare2 == combinatiiSiRezultat[i][0] and culoare1 == combinatiiSiRezultat[i][1] or culoare1 == combinatiiSiRezultat[i][0] and culoare2 == combinatiiSiRezultat[i][2] or culoare1 == combinatiiSiRezultat[i][2] and culoare2 == combinatiiSiRezultat[i][0] or culoare1 == combinatiiSiRezultat[i][1] and culoare2 == combinatiiSiRezultat[i][2] or culoare1 == combinatiiSiRezultat[i][2] and culoare2 == combinatiiSiRezultat[i][1]:
                    ok = 0
                    culoare_rezultata = combinatiiSiRezultat[i][2]
                    break
            elif culoare1 == culoare2 == combinatiiSiRezultat[i][0]:
                ok = 0
                culoare_rezultata = combinatiiSiRezultat[i][0]
                break
            elif culoare1 == culoare2 == combinatiiSiRezultat[i][1]:
                ok = 0
                culoare_rezultata = combinatiiSiRezultat[i][1]
                break
            elif culoare1 == culoare2 == combinatiiSiRezultat[i][2]:
                ok = 0
                culoare_rezultata = combinatiiSiRezultat[i][2]
                break
            elif culoare1 is None and culoare2 is not None:
                ok = 0
                culoare_rezultata = culoare2
                break
            elif culoare1 is not None and culoare2 is None:
                ok = 0
                culoare_rezultata = culoare1
                break

        if ok == 1:
            return 'nu'
        else:
            return culoare_rezultata
        

In [15]:
class Vas:
    def __init__(self, id, capacitate,ocupare,culoare):
        self.id = id
        self.capacitate = capacitate
        self.ocupare = ocupare
        self.culoare = culoare
    @staticmethod 
    def combina_vase(vas1, vas2):
        vasA = copy.deepcopy(vas1)
        vasB = copy.deepcopy(vas2)
        catAmTurnat = 0
        #print("\nVasA: " + str(vasA))
        #print("\nVasB "+ str(vasB) + "\n")
        ok = 1
        if vasB.ocupare == vasB.capacitate:
             ok = 1
        elif vasA.ocupare == 0 and vasB.ocupare==0:
            ok = 0
            
        elif vasA.ocupare == 0 and vasB.ocupare != 0:
            ok = 0
            
        elif vasA.ocupare != 0 and vasB.ocupare == 0:
                ok = 0
                catAmTurnat = 0
                if vasB.capacitate < vasA.ocupare:
                    catAmTurnat = vasB.capacitate
                    vasA.ocupare = vasA.ocupare - vasB.capacitate
                    vasB.ocupare = vasB.capacitate
                    vasB.culoare = vasA.culoare
                    return vasA, vasB, catAmTurnat
                elif vasB.capacitate >= vasA.ocupare:
                    catAmTurnat = vasA.ocupare
                    vasB.ocupare = vasB.ocupare + vasA.ocupare
                    vasA.ocupare = 0
                    vasB.culoare = vasA.culoare
                    if vasA.ocupare == 0:
                        vasA.culoare = ""
                    return vasA, vasB, catAmTurnat
        elif vasA.ocupare != 0 and vasB.ocupare != 0:
            if combina_culori(vasA.culoare, vasB.culoare) != 'nu':
                ok = 0
                if vasB.ocupare < vasB.capacitate:
                    if vasA.ocupare <= vasB.capacitate - vasB.ocupare:
                        catAmTurnat = vasA.ocupare
                        vasB.ocupare = vasB.ocupare + vasA.ocupare
                        vasA.ocupare = 0
                        vasB.culoare = combina_culori(vasA.culoare, vasB.culoare)
                        if vasA.ocupare == 0:
                            vasA.culoare = ""
                        return vasA, vasB, catAmTurnat
                    elif vasA.ocupare > vasB.capacitate - vasB.ocupare:
                        catAmTurnat = vasB.capacitate - vasB.ocupare
                        vasA.ocupare = vasA.ocupare - catAmTurnat
                        vasB.ocupare = vasB.capacitate
                        vasB.culoare = combina_culori(vasA.culoare, vasB.culoare)
                        return vasA, vasB, catAmTurnat
        if ok == 1:
            return "nu"
        return vasA, vasB, catAmTurnat
                        
    def __str__(self):
        if self.culoare !=  "":
            return "\nVasul" + str(self.id)+ " " + str(self.ocupare) +"/"+str(self.capacitate) + " " + self.culoare 
        else:
            return "\nVasul" + str(self.id)+ " " + str(self.ocupare)+ "/"+str(self.capacitate) + " " + "fara apa colorata"
    def __repr__(self):
        if self.culoare !=  "":
            return "\nVasul" + str(self.id)+ " " + str(self.ocupare) +"/"+str(self.capacitate) + " " + self.culoare 
        else:
            return "\nVasul" + str(self.id)+ " " + str(self.ocupare)+ "/"+str(self.capacitate) + " " + "fara apa colorata" 
    def __eq__(self, altu):
        return self.id == altu.id and self.capacitate == altu.capacitate and self.ocupare == altu.ocupare and self.culoare == altu.culoare
    


In [16]:
class Nod:
    def __init__(self, info, h):
        self.info = info
        self.h = h
        
        # prima euristica- consta in adunarea a numarului de litri din celelalte culori(anterioare) pe care trebuie sa le obtinem si numarul de litri de care
        # mai avem nevoie pentru a obtine nr cerut din culoarea curenta
        # daca depasim numarul de l de care avem nevoie, va trebui sa adaugam surplusul, semn ca trebuie gasita o cale mai eficienta si cu un cost mai mic
        # evitam cazul in care algoritmul creeaza prea multa culoare, existand riscul sa ramanem fara resurse pentru alte culori
    def calculeaza_h1(self):
        global stare_fin
        H = 0
        self.h = 0
        for x in stare_fin:
            sumaLitri = 0
            litriNecesari = x[0]
            culoareNecesara = x[1]
            for v in self.info:
                if v.culoare == culoareNecesara:
                    sumaLitri = sumaLitri + v.ocupare
            if sumaLitri == litriNecesari:
                self.h = self.h + 0      
            else:
                self.h = self.h + abs(litriNecesari - sumaLitri)
        return self.h
    
    # a doua euristica - daca nu am culoarea in starea initiala costul va creste deoarece va trebui formata, in rest bazandu-se pe prima euristica
    # in schimb, prioritizam crearea de culori noi
    def calculeaza_h2(self):
        global stare_fin
        self.h = 0
        for x in stare_fin:
            ok = 0
            sumaLitri = 0
            litriNecesari = x[0]
            culoareNecesara = x[1]
            for y in self.info:
                if y.culoare == culoareNecesara:
                    ok = 1
            if ok == 0:
                self.h = self.h + 10
            for v in self.info:
                if v.culoare == culoareNecesara:
                    sumaLitri = sumaLitri + v.ocupare
            if sumaLitri == litriNecesari:
                self.h = self.h + 0      
            else:
                self.h = self.h + abs(litriNecesari - sumaLitri)
        return self.h
        

        # euristica ce nu va duce niciodata la un rezultat presupune ca atunci cand turnam dintr-un vas in altul ea sa fie de fapt valoarea obtinuta, nu cat mai trebuie sa obtinem
        # spre exemplu daca eu trebuie sa obtin 5 l de apa mov, si am obtinut deja cei 5 l, euristica mea nu va mai fi 0 ca in cazurile anterioare, ci 5
        # in acest fel, stiu cu siguranta ca nu voi ajunge niciodata la o stare finala si programul va cicla
        # aceasta ciclare face euristica neadmisibila
    def calculeaza_eronat(self):
        global stare_fin
        H = 0
        self.h = 0
        for x in stare_fin:
            sumaLitri = 0
            litriNecesari = x[0]
            culoareNecesara = x[1]
            for v in self.info:
                if v.culoare == culoareNecesara:
                    sumaLitri = sumaLitri + v.ocupare
            if sumaLitri == litriNecesari:
                self.h = self.h + litriNecesari      
            else:
                self.h = self.h + sumaLitri
        return self.h
        
        
        
        
    def __str__(self):
        info = ""
        for vas in self.info:
            info+=str(vas)
        return "\n(vas= " + info + ", euristica= " + str(self.h) + ")" + "\n"
    def __repr__(self):
        info = ""
        for vas in self.info:
            info+=str(vas)
        return "\n(vas= " + info + ", euristica= " + str(self.h) + ")" + "\n"

In [17]:
class Arc:
    def __init__(self, start, end, cost):
        self.start= start;
        self.end = end
        self.cost = cost

In [18]:
class Problema:
    # imi initializez problema cu noduri si arce pt a putea parcurge 'graful' cu ponderi creat
    def __init__(self):
        global stare_init
        global stare_fin
        self.noduri =[stare_init]
        self.arce = []
        self.nod_start = stare_init
        self.nod_end  = stare_fin
        
    def cauta_nod_nume(self, info):
        for nod in self.noduri:
            ok = True
            for i in range(len(info)):
                if nod.info[i] != info[i]:
                    ok = False
            if ok is True:
                return nod
        return None
   

In [19]:
def citire(citesteDinFisierul):
       with open(citesteDinFisierul) as f:
        ok1 = 0
        ok2 = 0
        for linie in f:

            sir = linie.split()
            sir2 = ''.join(str(y) for y in sir)

            if sir2 != 'stare_init' and sir2 != 'stare_fin' and ok1 == 0 and ok2 == 0:
                posibileCombinatii.append((sir[0], sir[1]))
                combinatiiSiRezultat.append((sir[0], sir[1], sir[2]))
                setCulori.add(sir[0])
                setCulori.add(sir[1])
                setCulori.add(sir[2])
            elif sir2 == "stare_init":
                ok1 = 1
                continue
            elif sir2 == "stare_fin":
                ok2 = 1
                continue
            if ok1 == 1 and ok2 == 0:
                id = int(sir[0])
                capac = int(sir[1])
                ocupare = int(sir[2])
                if len(sir) == 4:
                    culoare = sir[3]
                    setCulori.add(sir[3])
                else:
                    culoare = ""
                
                stare_init.info.append(Vas(id, capac, ocupare, culoare))
                
            elif ok1==1 and ok2==1:
                ocupare = int(sir[0])
                culoare = sir[1]
                stare_fin.append((ocupare, culoare))
    
    

    

In [20]:
class NodParcurgere:
 
    problema =None
    def __init__(self, nod_graf ,succesori=[], parinte=None, g=0, f=None):
        self.nod_graf = nod_graf
        self.succesori = succesori
        self.parinte = parinte
        self.g = g
        if f is None:
            self.f = self.g + self.nod_graf.h
        else:
            self.f = f

    def drum_arbore(self):
        nod_c = copy.deepcopy(self)
        drum = [nod_c]
        while nod_c.parinte is not None:
            drum = [nod_c.parinte] + drum
            nod_c = nod_c.parinte
        return drum

    def contine_in_drum(self, nod):
        nod_c = self
        while nod_c.parinte is not None:
            if nod.info == nod_c.nod_graf.info:
                return True
            nod_c = nod_c.parinte
        return False


    def expandeaza(self):
        lista_succesori = []
        for i,x in enumerate(self.nod_graf.info):
            for j,y in enumerate(self.nod_graf.info):
                v = copy.deepcopy(x)
                w = copy.deepcopy(y)
                if i!=j and Vas.combina_vase(v,w) != 'nu' and v.ocupare != 0:
                        # print("\nVase combinate:\n")
                        # print("\nCuloare v: " + v.culoare + " si id: " + str(v.id ) + "\n")
                        # print("Culoare w: " + w.culoare + " si id: " + str(w.id ) + "\n")
                        # print("---------------------------------------------------------")
                        copie = copy.deepcopy(self.nod_graf.info)
                        k=0
                        copie[i], copie[j],k = Vas.combina_vase(v, w)
                        nod = Nod(copie, 0)
                        nod.h = nod.calculeaza_h2()
                        # nod.h = nod.calculeaza_eronat()
                        lista_succesori.append((nod, k))
        return lista_succesori
                

    def test_scop(self):
        sumaTotala = 0
        global stare_fin
        for x in stare_fin:
            sumaLitri = 0
            litriNecesari = x[0]
            culoareNecesara = x[1]
            for v in self.nod_graf.info:
                if v.culoare == culoareNecesara:
                    sumaLitri = sumaLitri + v.ocupare
            if sumaLitri == litriNecesari:
                sumaTotala = sumaTotala + 0      
            else:
                sumaTotala = sumaTotala + abs(litriNecesari - sumaLitri)
        if sumaTotala == 0:
            return True
        return False
      

    def __str__(self):
        parinte = self.parinte if self.parinte is None else self.parinte.nod_graf.info
        return "(nod: " + str(self.nod_graf) + ", parinte=" + str(parinte) + ", f=" + str(self.f) + ", g=" + str(
            self.g) + ")"

In [21]:
def afisare_rezultat(l):
    
    turnatIn = 0
    turnatDin = 0
    culoareTurnata = ""
    g.write("Lungimea drumului este: ")
    g.write(str(len(l)))
    g.write("\n==========================================\n")
    g.write("Drumul de cost minim:\n")
    g.write(str(l[0].nod_graf.info))
  
    for i in range(1, len(l)):
        for j in range(len(l[i].nod_graf.info)):
            if l[i].nod_graf.info[j].ocupare > l[i-1].nod_graf.info[j].ocupare:
                turnatIn = l[i].nod_graf.info[j].id
            elif l[i].nod_graf.info[j].ocupare < l[i-1].nod_graf.info[j].ocupare:
                culoareTurnata = l[i-1].nod_graf.info[j].culoare
                turnatDin = l[i-1].nod_graf.info[j].id
            
        g.write(" \n\n----Am turnat din vasul " + str(turnatDin) + "  " +str(l[i].g - l[i - 1].g) + " litri de apa de culoare " + culoareTurnata + " in vasul " +str(turnatIn)+  "----> \n\n" + str(l[i].nod_graf.info))


def in_lista(l, nod):
    for i in range(len(l)):
        if l[i].nod_graf.info == nod.info:
            return l[i]
    return None


def a_star():
    rad_arbore = NodParcurgere(NodParcurgere.problema.nod_start)
    open = [rad_arbore]
    closed = []

    if rad_arbore.test_scop() == True:
        g.write("\n------------------ Concluzie -----------------------\n")
        g.write("Solutia finala este aceeasi cu cea initiala")
        return 0
    while len(open) > 0:
  
        nod_curent = open.pop(0)
        closed.append(nod_curent)
        if nod_curent.test_scop():  
            break

        l_succesori = nod_curent.expandeaza()
        
        for (nod_succesor, cost_succesor) in l_succesori:
            
            if (not nod_curent.contine_in_drum(nod_succesor)):
                g_succesor = nod_curent.g + cost_succesor
                f = g_succesor + nod_succesor.h
                nod_parcg_vechi = in_lista(closed, nod_succesor)

                if nod_parcg_vechi is not None:
                    if (f < nod_parcg_vechi.f):
                        closed.remove(nod_parcg_vechi)
                        nod_parcg_vechi.parinte = nod_curent
                        nod_parcg_vechi.g = g_succesor
                        nod_parcg_vechi.f = f
                        nod_nou = nod_parcg_vechi
                else:
               
                    nod_parcg_vechi = in_lista(open, nod_succesor)
                    if nod_parcg_vechi is not None:
                        if (f < nod_parcg_vechi.f):
                            # if(nod_parcg_vechi.g>g_succesor):
                            open.remove(nod_parcg_vechi)
                            nod_parcg_vechi.parinte = nod_curent
                            nod_parcg_vechi.g = g_succesor
                            nod_parcg_vechi.f = f
                            nod_nou = nod_parcg_vechi
                    else:  #

                        nod_nou = NodParcurgere(nod_graf=nod_succesor, parinte=nod_curent,
                                                g=g_succesor)  
                if nod_nou:
                    
                    i = 0
                    while i < len(open):
                        if open[i].f < nod_nou.f:
                            i += 1
                        else:
                            while i < len(open) and open[i].f == nod_nou.f and open[i].g > nod_nou.g:
                                i += 1
                            break

                    open.insert(i, nod_nou)

    g.write("\n------------------ Concluzie -----------------------\n")
    if (len(open) == 0):
        g.write("Nu avem cum sa combinam vasele cu apa astfel incat sa ajungem la solutia finala")
    else:
        afisare_rezultat(nod_curent.drum_arbore())


In [22]:
setCulori = set()
posibileCombinatii = []
combinatiiSiRezultat = []
stare_init = Nod([], 0)
stare_fin = []

if __name__ == "__main__":
    
    x = time.time()
    CitesteDinFisierul = input()
    citire(CitesteDinFisierul)
    stare_init.calculeaza_h2()
    # stare_init.calculeaza_eronat()
    problema = Problema()
    NodParcurgere.problema = problema
    a_star()
    y = time.time()
    print("Timpul de executie al programului este ", y-x," secunde" )
    
   
   

 input.txt


Timpul de executie al programului este  5.003819465637207  secunde
